### Importing all the libraries

In [1]:
from skimage.io import imread
from skimage import color
from skimage.filters import threshold_otsu,gaussian
from skimage.transform import resize
import numpy
from skimage.segmentation import slic
from skimage.color import label2rgb

import tensorflow as tf
import numpy as np
import glob
from matplotlib import pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Loading the trained model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('NNmodel.h5')

In [ ]:
import pickle
pickle.dump(model, open("NNmodel.pkl", "wb"))

In [ ]:
import pickle
model3 = pickle.load(open('NNmodel.pkl', 'rb'))

### Loading the image file

In [ ]:
fig0 , ax0 = plt.subplots()
fig0.set_size_inches(10, 10)

imagename='PMI(172).jpg'
image=imread('./Test_set/PMI/'+imagename)

ax0.imshow(image)
plt.show()

### Removing gridlines / Perform Threshholding, Binarization, Gaussian filtering

In [ ]:
#importing gaussian filter and otsu threshold
from skimage.filters import threshold_otsu,gaussian
from skimage.transform import resize
import numpy

fig1 , ax1 = plt.subplots()

fig1.set_size_inches(10, 10)

#converting to gray scale
grayscale = color.rgb2gray(image)

#smoothing image
blurred_image = gaussian(grayscale, sigma=0.7)

#thresholding to distinguish foreground and background
#using otsu thresholding for getting threshold value
global_thresh = threshold_otsu(blurred_image)
print(global_thresh)

#creating binary image based on threshold
binary_global = blurred_image < global_thresh
binary_global = numpy.invert(binary_global)
ax1.imshow(binary_global,cmap='gray')
ax1.set_title("prossessed ECG")
ax1.axis('off')
image = binary_global

### Dividing the ECG leads from 1-13

In [ ]:
Lead_1 = image[300:600, 150:643]
Lead_2 = image[300:600, 646:1135]
Lead_3 = image[300:600, 1140:1625]
Lead_4 = image[300:600, 1630:2125]
Lead_5 = image[600:900, 150:643]
Lead_6 = image[600:900, 646:1135]
Lead_7 = image[600:900, 1140:1625]
Lead_8 = image[600:900, 1630:2125]
Lead_9 = image[900:1200, 150:643]
Lead_10 = image[900:1200, 646:1135]
Lead_11 = image[900:1200, 1140:1625]
Lead_12 = image[900:1200, 1630:2125]
Lead_13 = image[1250:1480, 150:2125]

Leads=[Lead_1,Lead_2,Lead_3,Lead_4,Lead_5,Lead_6,Lead_7,Lead_8,Lead_9,Lead_10,Lead_11,Lead_12,Lead_13]

In [ ]:
from skimage.segmentation import slic
from skimage.color import label2rgb

#plotting lead 1-12
fig2 , ax2 = plt.subplots(4,3)

fig2.set_size_inches(20, 20)

x_counter=0
y_counter=0

for x,y in enumerate(Leads[:len(Leads)-1]):
    if (x+1)%3==0:
        ax2[x_counter][y_counter].imshow(y, cmap="gray")
        ax2[x_counter][y_counter].axis('off')
        ax2[x_counter][y_counter].set_title("Lead {}".format(x+1))
        x_counter+=1
        y_counter=0
    else:
        ax2[x_counter][y_counter].imshow(y, cmap="gray")
        ax2[x_counter][y_counter].axis('off')
        ax2[x_counter][y_counter].set_title("Lead {}".format(x+1))
        y_counter+=1

#save preprocessed image
fig2.savefig(imagename, dpi=72)

### Importing the pre processed image

In [ ]:
image=tf.keras.preprocessing.image.load_img(imagename, color_mode='grayscale')
image=np.array(image)
image=np.invert(image)
print(image.shape)

### Reshape & Normalization

In [ ]:
x_mean = 2.73714811267664
x_std = 24.068791887587864
epsilon = 1e-10
input_reshaped = np.reshape(image, (1, 2073600))
input_norm = (input_reshaped - x_mean) / (x_std + epsilon)

### Prediction

In [ ]:
prediction = model.predict(input_norm)
pred = np.argmax(prediction[0])
print(pred)
if pred == 0:
    print('Normal')
elif pred == 1:
    print('Most likely to have an abnormal heartbeat')
elif pred == 2:
    print('Most likely to have a Myocardial infaraction')
elif pred == 3:
    print('Most likely to have a history in Myocardial infaraction')